## Coleta de Dados

In [1]:
# ETAPA: Coletar dados Iniciais (COLAB)
import pandas as pd
import numpy as np

print("--- Iniciando Coleta de Dados ---")
try:
    players = pd.read_csv("PicMoney-Base_Cadastral_de_Players.csv", encoding='latin-1', sep=';')
    transacoes = pd.read_csv("PicMoney-Base_de_Transa__es_-_Cupons_Capturados.csv", encoding='latin-1', sep=';')
    pedestres = pd.read_csv("PicMoney-Base_Simulada_-_Pedestres_Av__Paulista.csv", encoding='latin-1', sep=';')
    lojas = pd.read_csv("PicMoney-Massa_de_Teste_com_Lojas_e_Valores.csv", encoding='latin-1', sep=';')
    print("Sucesso! Os 4 arquivos CSV foram carregados.")
except FileNotFoundError:
    print("ERRO: Um ou mais arquivos CSV não foram encontrados.")

--- Iniciando Coleta de Dados ---
Sucesso! Os 4 arquivos CSV foram carregados.


## Descrição e Verificação da Qualidade dos Dados Brutos

In [2]:
# ETAPAS: Descrever os Dados e Verificar a Qualidade dos Dados (COLAB)
# Objetivo: Gerar um resumo estatístico para cada tabela antes da limpeza.

def resumo(df, nome_df):
    print(f"\n--- Resumo da Tabela '{nome_df}' (Dados Brutos) ---")
    resumo_df = pd.DataFrame({
        "Tipo de Dado": df.dtypes,
        "Valores Nulos (NA)": df.isna().sum(),
        "% Preenchido": (df.notna().mean() * 100).round(2),
        "Valores Distintos": df.nunique()
    })
    print(resumo_df)
    return resumo_df

# Gerando o resumo para cada uma das 4 tabelas
resumo_players = resumo(players, "Players")
resumo_transacoes = resumo(transacoes, "Transacoes")
resumo_pedestres = resumo(pedestres, "Pedestres")
resumo_lojas = resumo(lojas, "Lojas (Massa de Teste)")


--- Resumo da Tabela 'Players' (Dados Brutos) ---
                      Tipo de Dado  Valores Nulos (NA)  % Preenchido  \
celular                     object                   0        100.00   
data_nascimento             object                   0        100.00   
idade                        int64                   0        100.00   
sexo                        object                   0        100.00   
cidade_residencial          object                   0        100.00   
bairro_residencial          object                   0        100.00   
cidade_trabalho             object                3969         60.31   
bairro_trabalho             object                3969         60.31   
cidade_escola               object                6932         30.68   
bairro_escola               object                6932         30.68   
categoria_frequentada       object                   0        100.00   

                       Valores Distintos  
celular                             9999 

## Limpeza e Tratamento dos Dados

In [3]:
# ETAPA: Limpeza e Tratamento (COLAB)
# Objetivo: Aplicar as correções necessárias nos dados.

print("\n--- Iniciando Limpeza e Tratamento de Dados ---")

# Criando cópias para preservar os dataframes originais
players_clean = players.copy()
transacoes_clean = transacoes.copy()
lojas_clean = lojas.copy()

# Limpeza 'players_clean'
players_clean['data_nascimento'] = pd.to_datetime(players_clean['data_nascimento'], format='%d/%m/%Y', errors='coerce')
players_clean.fillna('Não Informado', inplace=True)

# Limpeza 'transacoes_clean'
transacoes_clean['data_hora'] = pd.to_datetime(transacoes_clean['data'] + ' ' + transacoes_clean['hora'], format='%d/%m/%Y %H:%M:%S', errors='coerce')
transacoes_clean.drop(columns=['data', 'hora'], inplace=True)

# Limpeza 'lojas_clean'
lojas_clean.rename(columns={'numero_celular': 'celular'}, inplace=True)
lojas_clean['data_captura'] = pd.to_datetime(lojas_clean['data_captura'], format='%d/%m/%Y', errors='coerce')
lojas_clean.dropna(subset=['valor_compra', 'valor_cupom'], inplace=True)

print("Limpeza concluída.")


--- Iniciando Limpeza e Tratamento de Dados ---
Limpeza concluída.


## Análise Exploratória (Relações de Dados)

In [4]:
# ETAPA: Explorar os Dados (COLAB)
# Objetivo: Unir as bases e gerar tabelas que mostram relações entre as variáveis.

print("\n--- Iniciando Análise e Relações dos Dados (com dados limpos) ---")

# Unindo a base de lojas (financeira) com a de players (demográfica)
df_analise_financeira = pd.merge(lojas_clean, players_clean, on='celular', how='left')
df_analise_financeira.dropna(subset=['idade'], inplace=True)
df_analise_financeira['faixa_etaria'] = pd.cut(df_analise_financeira['idade'], bins=[0, 18, 25, 35, 50, 65, 100], labels=['0-18', '19-25', '26-35', '36-50', '51-65', '65+'], right=False)

# Relação 1: Gasto Médio (Ticket Médio) por Faixa Etária
print("\n--> Relação: Gasto Médio por Compra (Ticket Médio) por Faixa Etária")
print(df_analise_financeira.groupby('faixa_etaria')['valor_compra'].mean().round(2))

# Relação 2: Análise Financeira por Tipo de Cupom
print("\n--> Relação: Análise Financeira por Tipo de Cupom")
print(df_analise_financeira.groupby('tipo_cupom').agg(Receita_Total=('valor_compra', 'sum'), Custo_Total_Cupons=('valor_cupom', 'sum')))


--- Iniciando Análise e Relações dos Dados (com dados limpos) ---

--> Relação: Gasto Médio por Compra (Ticket Médio) por Faixa Etária
faixa_etaria
0-18        NaN
19-25       NaN
26-35    249.27
36-50       NaN
51-65    307.22
65+         NaN
Name: valor_compra, dtype: float64

--> Relação: Análise Financeira por Tipo de Cupom
            Receita_Total  Custo_Total_Cupons
tipo_cupom                                   
Cashback           249.27               12.46
Desconto           307.22               77.18


/tmp/ipython-input-1711791396.py:13: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  print(df_analise_financeira.groupby('faixa_etaria')['valor_compra'].mean().round(2))
